This notebook contains some basic tests to check the model and limit setting is working correctly.

In [1]:
import numpy as np
from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rc('font', size=16)
plt.rcParams['figure.figsize'] = (12.0, 10.0)    # resize plots

from wimpy.model import Model
from wimpy.models.temp import config as base_config
from wimpy.analysis import bestfit, interval, loglikelihood

In [2]:
# Create the model from scratch (takes a minute or more to sample the PDFs)
m = Model(base_config, pdf_sampling_multiplier=0.5)
m.save('cs1_model')

# The next time you can just load it from disk, like so:
# m = Model.load('cs1_model')

'cs1_model'

# Basic diagnostics

Energy spectra of sources included in model

In [ ]:
for s in m.sources:
    s.energy_distribution.plot(label=s.label, color=s.color)
plt.yscale('log')

plt.ylabel('Events /day/kg/keV')
plt.xlabel("Energy (keV)")
plt.legend(loc='center right')
plt.ylim(1e-11, 1e-2)

What fraction of the total events produced an event inside the analysis space (as opposed to an event out of range or no event at all)?

In [4]:
print([(s.name, s.fraction_in_range) for s in m.sources])

[('er_bg', 0.52596036363636367), ('cnns', 0.00039666666666666664), ('radiogenics', 0.35388066666666668), ('wimp_50gev', 0.63900433333333329)]


Simulate a single toy data set with "WIMP strength -1" i.e. a cross section of 10^(-46) cm^2. 

In [ ]:
truth = -1
d = m.simulate(truth)
m.show(d)
plt.legend(loc='lower right', scatterpoints=1, markerscale=2)

Get the best-fit strength, without and with considering a 30% rate uncertainty on the 50 GeV WIMP signal

(10x the one applied in the Bologna model for Leff, see [here](https://xecluster.lngs.infn.it/dokuwiki/doku.php?id=xenon:xenon1t:sim:notes:digangi:leff_impact_on_pl), just to show an effect)

In [6]:
bestfit(m, d)

(array(-1.1234008941548541), -5874.2274721138683)

In [7]:
m.sources[-1].rate_uncertainty = 0.3
bestfit(m, d, fit_uncertainties=True)

(array([-1.12262584, -0.00594011]), -5874.2274897563138)

Show the likelihood and profile likelihood curves

In [8]:
bestfit(m, guess_strength=1, d=d, fit_uncertainties=True, fit_strength=False)

(array(-3.3081476958191884), -5879.6995530718586)

In [ ]:
strengths = np.linspace(-5, 0, 200)
lls = np.array([loglikelihood(m, wimp_strength=x, d=d) for x in tqdm(strengths)])
max_ll = lls.max()
plt.axvline(strengths[np.argmax(lls)], label='ML estimate', c='red', linestyle='--')
plt.plot(strengths, (max_ll-lls), label='LR')

plls = np.array([bestfit(m, guess_strength=x, d=d, fit_uncertainties=True, fit_strength=False)[1]
                 for x in tqdm(strengths)])
max_pll = plls.max()
plt.axvline(strengths[np.argmax(plls)], label='pL estimate', c='purple', linestyle='-.')
plt.plot(strengths, (max_pll-plls), label='Profile LR')

plt.xlabel('Log10(WIMP cross section (zb))')
plt.ylabel('- Log likelihood ratio')
plt.axvline(truth, label='Truth', c='r', linestyle=':')
plt.legend(loc='upper left')
plt.ylim(0, 5)
plt.xlim()
plt.show()

# p_ratio_hist(m).plot(vmin=1e-6, vmax=1e4, log_scale=True,  

#                      cblabel='s/b PDF ratio', cmap=plt.cm.Greens)
# m.show(d)
# plt.show()

Give 90% confidence intervals and 90% upper limits on the WIMP cross section:

In [10]:
interval(m, d, kind='limit')

-1.0110296573075004

In [11]:
low, high = interval(m, d, kind='central') 
print(low, high, high-low)

-1.2932088154801475 -0.9816434751189275 0.3115653403612201


Same with profiling. The limit should get worse (ie. higher, ie less negative) and the interval should get broader:

In [12]:
interval(m, d, kind='limit', profile=True)

-0.8880154826835885

In [13]:
low, high = interval(m, d, kind='central', profile=True)
print(low, high, high-low)

-1.3701638361277713 -0.8016803173813319 0.5684835187464393


(If I try this for a rate uncertainty on the ER background, I get only a minute effect. Actually it seems the interval gets narrowed, athough minisculely so...)

Compute the sensitivity, for comparison against the Bologna model results:

In [14]:
from scipy import stats
def get_sensitivity(model, n_trials=1000, profile=False):
    bg_limits = [interval(model, 
                          model.simulate(wimp_strength=model.no_wimp_strength), 
                          kind='limit', 
                          profile=profile)
                 for _ in tqdm(range(n_trials))]
    med = np.median(bg_limits)
    ll = np.percentile(bg_limits, stats.norm.cdf(-1) * 100)
    ul = np.percentile(bg_limits, stats.norm.cdf(1) * 100)
    print(med, (ul-ll)/2/np.sqrt(n_trials))
    #return [10**x * reference_wimp_cross_section for x in (med, ll, ul)]
    
get_sensitivity(m)

-2.0498500989 0.00998188337905


To compare against the Bologna model, consider that
  * CLS is a factor 1.4
  * LEff is a very small factor at high/intermediate wimp masses, see[here](https://xecluster.lngs.infn.it/dokuwiki/lib/exe/detail.php?id=xenon%3Axenon1t%3Asim%3Anotes%3Adigangi%3Asensitivity-profile-likelihood-ratio&media=xenon:xenon1t:sim:notes:digangi:csensitivity_ratio.png))

Fortunately we have the sensitivity without CLS (but with LEff) [here](https://xecluster.lngs.infn.it/dokuwiki/doku.php?id=xenon:xenon1t:sim:notes:digangi:sensitivity-profile-likelihood-ratio#with_or_without_cls): 8.6e-48 cm^2. Expressed in the same units, this is:

In [15]:
np.log10(8.6*10**-48)+45

-2.0655015487564299